## Summary

----

## Imports

In [1]:
import copy
import os
import time
from pathlib import Path

import nglview
import numpy as np
import pandas as pd
from IPython.display import Image

import matplotlib.pyplot as plt
import proteinsolver
from kmbio import PDB

_ColormakerRegistry()

/home/strokach/env/lib/python3.7/site-packages/Bio/KDTree/__init__.py:27: BiopythonDeprecationWarning: Bio.KDTree has been deprecated, and we intend to remove it in a future release of Biopython. Please use Bio.PDB.kdtrees instead, which is functionally very similar.
  BiopythonDeprecationWarning,


In [2]:
pd.set_option("max_row", 1000)
pd.set_option("max_column", 1000)

## Parameters

In [3]:
NOTEBOOK_NAME = "16_david_analysis_quark"
NOTEBOOK_PATH = Path(NOTEBOOK_NAME).resolve()
NOTEBOOK_PATH.mkdir(exist_ok=True)
NOTEBOOK_PATH

PosixPath('/home/kimlab1/strokach/workspace/proteinsolver/notebooks/16_david_analysis_quark')

In [4]:
# STRUCTURE_ID = "1n5u"  # poster
# STRUCTURE_ID = "4z8j"
STRUCTURE_ID = "4unu"
# STRUCTURE_ID = "4beu"  # poster

STRUCTURE_ID

'4unu'

In [5]:
STRUCTURE_NAMES = {
    #
    "1n5u": "1n5uA03",
    "4beu": "4beuA02",
    "4z8j": "4z8jA00",
    "4unu": "4unuA00",
}

STRUCTURE_FILE = Path(proteinsolver.__path__[0]).joinpath("data", "inputs", f"{STRUCTURE_NAMES[STRUCTURE_ID]}.pdb")
assert STRUCTURE_FILE.is_file()

In [6]:
MODEL_SUBFOLDERS = {
    #
    "1n5u": "QA4879",
    "4beu": "QA4880",
    "4z8j": "QA4902",
    "4unu": "QA4893",
}

MODEL_FILE = NOTEBOOK_PATH.joinpath("Quark", STRUCTURE_ID, MODEL_SUBFOLDERS[STRUCTURE_ID], "model1.pdb")

In [7]:
ORIENTATIONS = {
    "1n5u": [
        43.000235831433194,
        -5.550381294767174,
        -32.49843992783927,
        0,
        25.444615585674025,
        39.5510614767004,
        26.91209180987381,
        0,
        20.96489608745896,
        -36.61806132728099,
        33.99362187959817,
        0,
        -35.0819034576416,
        -9.174790382385254,
        -4.088220119476318,
        1,
    ],
    "4z8j": [
        6727725260488,
        13.053016191055,
        0,
        -31.621760287315702,
        15.795202008820427,
        59.95156556753002,
        0,
        38.35759631472927,
        -47.88805958313651,
        32.84878838684845,
        0,
        3.4756088256835938,
        -5.889947414398193,
        6.526662826538086,
        1,
    ],
    "4z8j": [
        38.56395789892707,
        -2.475623293770176,
        52.77879474940134,
        0,
        27.519522419952263,
        56.72017890850758,
        -17.447232591065386,
        0,
        -45.10438749910903,
        32.49001890760264,
        34.48045293258368,
        0,
        -8.420963764190674,
        -3.637119770050049,
        -16.13066938519478,
        1,
    ],
    "4beu": [
        56.1060851095829,
        31.380589143088347,
        60.173765033863994,
        0,
        9.705000472391331,
        73.5625642506499,
        -47.41179459102586,
        0,
        -67.1672266745388,
        36.84187733467765,
        43.413775405387696,
        0,
        -15.809061646461487,
        -23.35176646709442,
        -18.411685943603516,
        1,
    ],
}

## View structures

In [8]:
structure_all = PDB.load(STRUCTURE_FILE)
structure = PDB.Structure(STRUCTURE_FILE.name + "A", structure_all[0].extract('A'))
assert len(list(structure.chains)) == 1

In [9]:
model = PDB.load(MODEL_FILE)
assert len(list(model.chains)) == 1

In [10]:
view = PDB.view_structure(structure)
view.add_component(PDB.structure_to_ngl(model))

view

NGLWidget()

In [11]:
model_aligned = copy.deepcopy(model)

if STRUCTURE_ID == "4beu":
    i = 0
    structure_slice = slice(i, -17 + i)
else:
    structure_slice = slice(None, None)

fixed = [a for r in list(structure[0]["A"])[structure_slice] for a in r if a.id in ["N", "CA", "C", "O"]]
moving = [a for r in model[0][" "] for a in r if a.id in ["N", "CA", "C", "O"]]

sup = PDB.Superimposer()
sup.set_atoms(fixed, moving)
sup.apply(list(model_aligned.atoms))

distance = np.linalg.norm(
    np.array([a.coord for r in list(structure.residues)[structure_slice] for a in r if a.id in ["N", "CA", "C", "O"]])
    - np.array([a.coord for a in model_aligned.atoms if a.id in ["N", "CA", "C", "O"]])
)
print(distance)

34.52254929272022


In [12]:
view = PDB.view_structure(model_aligned)
view.add_component(PDB.structure_to_ngl(structure))

view.component_0.clear_representations()
view.component_0.add_representation("cartoon", color="white")

view.component_1.clear_representations()
view.component_1.add_representation("cartoon")

if STRUCTURE_ID in ORIENTATIONS:
    view._remote_call("orient", target="viewerControls", args=(ORIENTATIONS[STRUCTURE_ID],))

view

NGLWidget()

In [13]:
time.sleep(2)
img = view.render_image(factor=5, antialias=True, trim=True, transparent=True)
time.sleep(2)

In [14]:
screenshot_file = NOTEBOOK_PATH.joinpath(f"quark-aligned-{STRUCTURE_ID}.png")
print(screenshot_file)

with screenshot_file.open("wb") as fout:
    fout.write(img.value)

/home/kimlab1/strokach/workspace/proteinsolver/notebooks/16_david_analysis_quark/quark-aligned-4unu.png


In [16]:
view.get_state()["_camera_orientation"]

[38.56395789892707,
 -2.475623293770176,
 52.77879474940134,
 0,
 27.519522419952263,
 56.72017890850758,
 -17.447232591065386,
 0,
 -45.10438749910903,
 32.49001890760264,
 34.48045293258368,
 0,
 -8.420963764190674,
 -3.637119770050049,
 -16.13066938519478,
 1]